<a href="https://colab.research.google.com/github/ashibullah/Romanian-Bangla-Sentiment-Analysis-NLP/blob/main/RomanianBanglaSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets --upgrade

In [2]:
from datasets import load_dataset

In [ ]:
df = load_dataset("aplycaebous/BnSentMix" , split = "train")

In [4]:
from textblob import TextBlob
import re

In [5]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [6]:
def correct_text(text):
    return str(TextBlob(text).correct())

In [7]:
print(df[20])

{'Sentence': 'Vloi bt cash out korte 15 takar jaigai 18.50 paisa nei', 'Label': 3}


In [ ]:
import pandas as pd
# Create DataFrame
df = pd.DataFrame(df)
# Convert 'Sentence' column to lowercase
df['Sentence'] = df['Sentence'].str.lower()
print(df)

In [ ]:
# remove urls
df['Sentence'] = df['Sentence'].apply(clean_text)
print(df)

In [ ]:
df = df['Sentence'].apply(correct_text)
# avoiding this